In [111]:
import pandas as pd
import numpy as np

In [112]:
# Step 1: Load the Datasets ---
# Load the first file
df_internet = pd.read_csv("Households - level of internet access.csv")

# Load the second file
df_remote = pd.read_csv("Remote_work_in_Romania.csv")

In [113]:
# Step 2: Cleaning the Remote Work DataFrame --
# Rename columns for easier access (removing special characters and spaces).
df_remote.rename(columns={
    'Country': 'Country',
    '2022 (%)': 'Remote_Work_2022',
    '2024 (%)': 'Remote_Work_2024'
}, inplace=True)

In [114]:
# We can drop 'Country Code' as we will merge by the full country name.
df_remote.drop(columns=['Country Code'], inplace=True)

In [115]:
# Convert percentage columns to a numeric type (float).
# The 'coerce' option will automatically turn any non-numeric values into NaN (Not a Number),
# which is Pandas' standard for a null value.
for col in ['Remote_Work_2022', 'Remote_Work_2024']:
    df_remote[col] = pd.to_numeric(df_remote[col], errors='coerce')

In [116]:
# Step 3: Cleaning the Internet Access DataFrame --
# Rename columns to match the new, cleaner format.
df_internet.rename(columns={
    'Country': 'Country',
    '2022 (%)': 'Internet_Access_2022',
    '2024 (%)': 'Internet_Access_2024'
}, inplace=True)

In [117]:
df_internet.drop(columns=['Country Code'], inplace=True)

In [118]:
# Convert percentage columns to a numeric type (float).
# The 'coerce' option will automatically turn any non-numeric values into NaN (Not a Number),
# which is Pandas' standard for a null value.
for col in ['Internet_Access_2022', 'Internet_Access_2024']:
    df_internet[col] = pd.to_numeric(df_internet[col], errors='coerce')

In [119]:
# Step 4: Merge the Datasets ---
# We will perform an 'inner' merge on the 'Country' column.
# This ensures that we only keep countries that exist in BOTH datasets.
df_merged = pd.merge(df_remote, df_internet, on='Country', how='inner')

In [120]:
df_merged

,Country,Remote_Work_2022,Remote_Work_2024,Internet_Access_2022,Internet_Access_2024
0,European Union - 27 countries (from 2020),49.98,52.90,92.44,94.19
1,Belgium,63.49,70.06,94.44,94.62
2,Bulgaria,28.25,32.43,87.31,92.12
3,Czechia,42.04,46.60,91.48,94.56
4,Denmark,78.00,78.46,95.16,96.92
5,Germany,60.45,61.03,91.41,92.65
6,Estonia,47.71,57.71,92.41,92.89
7,Ireland,63.66,57.56,NaN,92.97
8,Greece,32.94,41.93,85.49,86.89
9,Spain,50.68,55.09,96.08,96.83


In [121]:
# Step 5: Feature Engineering: Adding New, Relevant Columns ---
# Calculate the percentage point change in remote work adoption.
df_merged['Remote_Work_Change'] = df_merged['Remote_Work_2024'] - df_merged['Remote_Work_2022']

# Calculate the percentage point change in household internet access.
df_merged['Internet_Access_Change'] = df_merged['Internet_Access_2024'] - df_merged['Internet_Access_2022']

In [122]:
# Create a categorical 'Trend' column to easily identify growth or decline in remote work.
def get_trend(change):
    if change > 1:
        return 'Growth'
    elif change < -1:
        return 'Decline'
    elif -1 <= change <= 1:
        return 'Stable'
    else:
        return 'Unknown'

df_merged['Remote_Work_Trend'] = df_merged['Remote_Work_Change'].apply(get_trend)

In [123]:
# Step 6: Final Data Inspection and Handling Missing Values ---
print("Final data inspection...")
# Display the data types and non-null counts for all columns.
print("DataFrame Info:")
df_merged.info()
print("\n")

# Check for missing values in the DataFrame.
print("Missing Values:")
print(df_merged.isnull().sum())
print("\n")

Final data inspection...
DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Country                 35 non-null     object 
 1   Remote_Work_2022        35 non-null     float64
 2   Remote_Work_2024        33 non-null     float64
 3   Internet_Access_2022    32 non-null     float64
 4   Internet_Access_2024    35 non-null     float64
 5   Remote_Work_Change      33 non-null     float64
 6   Internet_Access_Change  32 non-null     float64
 7   Remote_Work_Trend       35 non-null     object 
dtypes: float64(6), object(2)
memory usage: 2.3+ KB


Missing Values:
Country                   0
Remote_Work_2022          0
Remote_Work_2024          2
Internet_Access_2022      3
Internet_Access_2024      0
Remote_Work_Change        2
Internet_Access_Change    3
Remote_Work_Trend         0
dtype: int64




In [124]:
# Step 7: As a final step, let's sort the data by the 2024 remote work percentage
# to make it easier to analyze the rankings.
df_final_sorted = df_merged.sort_values(by='Remote_Work_2024', ascending=False).reset_index(drop=True)

In [125]:
print("Displaying the head of the final, prepared DataFrame:")
print(df_final_sorted.head())

Displaying the head of the final, prepared DataFrame:
   Country  Remote_Work_2022  Remote_Work_2024  Internet_Access_2022  \
0  Finland             78.50             84.54                 97.59   
1   Norway             76.96             80.46                 99.01   
2   Sweden             79.44             79.07                 94.33   
3  Denmark             78.00             78.46                 95.16   
4  Belgium             63.49             70.06                 94.44   

   Internet_Access_2024  Remote_Work_Change  Internet_Access_Change  \
0                 97.35                6.04                   -0.24   
1                 98.10                3.50                   -0.91   
2                 95.43               -0.37                    1.10   
3                 96.92                0.46                    1.76   
4                 94.62                6.57                    0.18   

  Remote_Work_Trend  
0            Growth  
1            Growth  
2            Stable 

In [126]:
# Round mathematically Remote_Work_Change and Internet_Access_Change to have only 2 decimal places.
df_final_sorted['Remote_Work_Change'] = df_final_sorted['Remote_Work_Change'].round(2)
df_final_sorted['Internet_Access_Change'] = df_final_sorted['Internet_Access_Change'].round(2)

In [127]:
# Optionally, save the prepared data to a new CSV file for the next steps.
df_final_sorted.to_csv('romania_remote_work_prepared_data.csv', index=False)
print("\nFinal DataFrame saved to 'romania_remote_work_prepared_data.csv'")


Final DataFrame saved to 'romania_remote_work_prepared_data.csv'
